In [1]:
import time

#Takes in:
#- df_preds, your predictions in the format required for submission to kaggle
#- df_actual, a dataframe that contains target bool values AND srch_ids 
#             (ie Test set that contains X and y)

def NDCG(df_preds, df_actual):
    start = time.time()
    IDCG_per_id = []
    DCG_5_per_id = []
    
    ids = df_preds['srch_id'].unique()
    
    # For each user, score each of the user's query
    score_index = -1
    
    #### For printing TIME #####
    ids_length = len(ids)
    t = 0
    start = time.time()
    ############################
    
    for item in ids: 
        t+=1
        if not t%1000:
            print(f"{t}/{ids_length} ids passed: Time elapsed: {time.time() - start}")
        
        ####Printing time #####
        #t+=1
        #if t%100 == 0:
        #    print(f" ID {t}/{ids_length}, time elapsed: {time.time() - start}")
        #######################
        
        score_index += 1
        IDCG_per_id.append(0)
        DCG_5_per_id.append(0)
        
        # chunk of submission dataframe belonging to particular user:
        preds = df_preds[df_preds['srch_id'] == item]
        # chunk of true dataframe belonging to particular user:
        real = df_actual[df_actual['srch_id'] == item]
        #for each occurence of user ID:
        
        for i in range(0,len(real[real['srch_id'] == item])):
            
            score = 0
            #property id from a row from submission df
            prop_pred = preds.iloc[i][1]
            
            #Get score for current prop_id
            if (real[real['prop_id'] == prop_pred]['booking_bool'] == 1).iloc[0]:
                score = 5
            elif (real[real['prop_id'] == prop_pred]['click_bool'] == 1).iloc[0]:
                score = 1
            else:
                score = 0
            
            #apply NDCG to score
            DCG_score = score/(np.log2((i+1)+1))
            IDCG_per_id[score_index] += DCG_score
            
            if i <= 4:
                #print(f"i is {i} and adding {DCG_score} to DCG_5")
                DCG_5_per_id[score_index] += DCG_score


    print(f"Total Time: {time.time() - start}")
    

    # Calculat Average NDCG_5 score over all unique srch_ids
    NDCG_5 = 0
    for i in range(0,len(ids)):
        IDCG = IDCG_per_id[i]
        DCG_5 = DCG_5_per_id[i]
        NDCG_5 += DCG_5/IDCG
        
    
    NDCG_5 = NDCG_5/len(ids)
        
    
    print(f"Average score (NDCG_5 score) is: {NDCG_5}")
    return(IDCG_per_id,DCG_5_per_id)